In [1]:
import pandas as pd

pre_separated_data = True

In [2]:
df1_path = "/data/challenges/hackathon_bigdata_2025/raw_data/part-00000-tid-2779033056155408584-f6316110-4c9a-4061-ae48-69b77c7c8c36-4-1-c000.snappy.parquet"
df2_path = "/data/challenges/hackathon_bigdata_2025/raw_data/part-00000-tid-5196563791502273604-c90d3a24-52f2-4955-b4ec-fb143aae74d8-4-1-c000.snappy.parquet"
df3_path = "/data/challenges/hackathon_bigdata_2025/raw_data/part-00000-tid-6364321654468257203-dc13a5d6-36ae-48c6-a018-37d8cfe34cf6-263-1-c000.snappy.parquet"

In [3]:
df = pd.read_parquet(df1_path)
df2 = pd.read_parquet(df2_path)

# Df3 é bem pesado (vai lotar a memoria, acho que precisamos usar o polars aqui)
#df3 = pd.read_parquet("/data/challenges/hackathon_bigdata_2025/raw_data/part-00000-tid-6364321654468257203-dc13a5d6-36ae-48c6-a018-37d8cfe34cf6-263-1-c000.snappy.parquet")

In [4]:
if not pre_separated_data:
    import polars as pl

    df3 = pl.read_parquet(df3_path)
    print(df3.shape)

    total_rows = df3.shape[0]
    chunk_size = total_rows // 10  # Integer division to get the chunk size

    for i in range(10):
        start_index = i * chunk_size
        end_index = start_index + chunk_size
        
        if i == 9:
            chunk = df3.slice(start_index)
        else:
            chunk = df3.slice(start_index, chunk_size)

        output_filename = df3_path.strip(".snappy.parquet") + f"_part_{i+1}.parquet"
        
        chunk.write_parquet(output_filename)
        print(f"Written {output_filename}")

In [5]:
if pre_separated_data:
    all_dfs = []
    for i in range(10):
        df3 = pd.read_parquet(df3_path.strip(".snappy.parquet") + f"_part_{i+1}.parquet")
        all_dfs.append(df3)
    df3 = pd.concat(all_dfs, ignore_index=True)
    del all_dfs

In [6]:
df3

,produto,categoria,descricao,tipos,label,subcategoria,marca,fabricante
0,69753381296344216,Package,MONDAY NIGHT BREW DON'T CALL IT HOTLANTA 6/4/1...,Package,In&Out,Red Wine,Monday Night Don't Call It Hotlanta,Monday Night Brewing
1,5019314249828979377,Distilled Spirits,BARRELL V907 RED PHONE BOOTH & TOWER,Distilled Spirits,Specialty,Lager / Pilsner,Barrell Bourbon- Private Label,Barrell Craft Spirits
2,4016404282141162328,Distilled Spirits,DRINKWORKS SIMPLY REFRESHING GIN & TONIC 12/4/...,Distilled Spirits,Core,Lager,Drinkworks Simply Refreshing Gin & Tonic,AB Drinkworks
3,6217366559810422145,Distilled Spirits,CENZON REPOSADO 100% AGAVE 6/750ML 80PF,Distilled Spirits,None,Lager,Cenzon Reposado 100% Agave Tequila,Sazerac Spirits
4,7356488787409434558,Draft,REFORMATION STARK 1/4 KEG,Draft,Core,Red Wine,Reformation Stark,Reformation Brewery
...,...,...,...,...,...,...,...,...
192356311,2910976334046702887,ABA Spirits,SAZERAC OLD WESTBURY GIN WNBS 1LTR 42PRF,Wine > 14 %,Core,Lager,Old Westbury Extra Dry Gin,Sazerac ABA
192356312,997545005038320324,Package,TX RANCH WATER LIME 4/6 CN,Package,Core,Soda,TX Ranch Water Lime,MPL Brands NV/Patco
192356313,5342092316314076162,Draft,TANTRUM POEMS AT MIDNIGHT RASPBERRY BLACKBERRY...,Draft,Discontinued,Energy,Tantrum Poems at Midnight Raspberry Blackberry,Tantrum Brewing Co
192356314,8663495453589323322,Wine,SALVESTRIN RETTAGIO 2019,Wine > 14 %,Close Out,Lager,Salvestrin Rettagio,Salvestrin Wine Company


In [7]:
df.columns

Index(['pdv', 'premise', 'categoria_pdv', 'zipcode'], dtype='object')

In [8]:
df2.columns

Index(['internal_store_id', 'internal_product_id', 'distributor_id',
       'transaction_date', 'reference_date', 'quantity', 'gross_value',
       'net_value', 'gross_profit', 'discount', 'taxes'],
      dtype='object')

In [10]:
df3.columns

Index(['produto', 'categoria', 'descricao', 'tipos', 'label', 'subcategoria',
       'marca', 'fabricante'],
      dtype='object')

In [11]:
df.head()

,pdv,premise,categoria_pdv,zipcode
0,2204965430669363375,On Premise,Mexican Rest,30741
1,5211957289528622910,On Premise,Hotel/Motel,80011
2,9024493554530757353,Off Premise,Convenience,80751
3,8659197371382902429,On Premise,Restaurant,80439
4,1400854873763881130,On Premise,Restaurant,30093


In [12]:
df.pdv.unique().shape # Número de PDVs únicos

(14419,)

In [17]:
df.zipcode.unique().shape # Número de Zipcodes únicos

(788,)

In [13]:
df.premise.value_counts()

premise
On Premise     8216
Off Premise    6203
Name: count, dtype: int64

In [15]:
df.categoria_pdv.value_counts()

categoria_pdv
Restaurant                   3316
Convenience                  2849
Package/Liquor               2153
Bar                          1444
Grocery                       697
Mexican Rest                  623
Other On Premise              473
Hotel/Motel                   359
Asian                         263
Pizza                         187
Super Center                  181
Golf - Public                 168
Drug                          167
Italian                       144
Service Org                   138
Special Event                 137
Golf - Private                113
Billiard/Bowling               91
Sports/Rec Club                88
Stadium/Concession             79
Barbeque                       76
Theatre                        75
Club Store                     65
Banquet/Caterer                55
Night Club                     52
Irish                          46
Church                         43
Sample Room                    35
Music Venue                    33


In [19]:
df2.head()

,internal_store_id,internal_product_id,distributor_id,transaction_date,reference_date,quantity,gross_value,net_value,gross_profit,discount,taxes
0,7384367747233276219,328903483604537190,9,2022-07-13,2022-07-01,1.0,38.125000,37.890625,10.042625,3.950000,0.234375
1,3536908514005606262,5418855670645487653,5,2022-03-21,2022-03-01,6.0,107.250000,106.440002,24.732002,17.100000,0.810000
2,3138231730993449825,1087005562675741887,6,2022-09-06,2022-09-01,3.0,56.625000,56.220001,14.124002,5.250000,0.405000
3,3681167389484217654,1401422983880045188,5,2022-09-11,2022-09-01,129.0,1037.160023,1037.160023,156.348026,479.880006,0.000000
4,7762413312337359369,6614994347738381720,4,2022-02-18,2022-02-01,1.0,26.230000,23.950241,6.550241,0.000000,2.279758


In [22]:
df.shape, df2.shape

((14419, 4), (6560698, 11))

In [26]:
df2 = df2.merge(df, how="inner", left_on="internal_store_id", right_on="pdv")

In [27]:
df2.isna().sum()

internal_store_id      0
internal_product_id    0
distributor_id         0
transaction_date       0
reference_date         0
quantity               0
gross_value            0
net_value              0
gross_profit           0
discount               0
taxes                  0
pdv_x                  0
premise_x              0
categoria_pdv_x        0
zipcode_x              0
pdv_y                  0
premise_y              0
categoria_pdv_y        0
zipcode_y              0
dtype: int64

In [29]:
df2.duplicated().sum(), df2.shape

(np.int64(0), (6515116, 19))

In [33]:
df3.merge(df2, how="left", left_on="produto", right_on="internal_product_id")
#df2.merge(df3, how="inner", left_on="internal_product_id", right_on="produto").shape

MemoryError: Unable to allocate 1.29 TiB for an array with shape (176710087974,) and data type int64

In [35]:
df3.iloc[0:100000].merge(df2, how="inner", left_on="produto", right_on="internal_product_id")

KeyboardInterrupt: 